In [1]:
#### Install Dependencies for library ####
install.packages("magrittr")
install.packages("data.table")
install.packages("ggplot2")
install.packages("gplots")
install.packages("BiocManager")
library("BiocManager")
BiocManager::install("WGCNA")
install.packages("DGCA")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘gtools’, ‘caTools’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.16 (BiocManager 1.30.19), R 4.2.2 Patched (2022-11-10
  r83330)

Installing package(s) 'BiocVersion', 'WGCNA'

also installing the dependencies ‘zlibbioc’, ‘RCurl’, ‘GenomeInfoDbData’, ‘deldir’, ‘RcppEigen’, ‘XVector’, ‘GenomeInfoDb’, ‘png’, ‘jpeg’, ‘interp’, ‘checkmate’, ‘htmlwidgets’, ‘plogr’, ‘Biostrings’, ‘Formula’, ‘latticeExtra

In [1]:
library(magrittr)
library(data.table)
library(ggplot2)
library(gplots)
library(WGCNA)
library(DGCA)


Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess


Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor




In [2]:
### downloading data
clinical_patient_url <- "https://drive.google.com/uc?id=1vHyUxYv9Tmk2Pa_TR49NZ0IsKIR1WzWb&export=download"
clinical_sample_url <- "https://drive.google.com/uc?id=1w6lGMtvoPAQfV4rHYSa6loo8bVdTrRiy&export=download"
gbm_url <- "https://drive.google.com/uc?id=1_4P14tcl4U2VMKUIaZcUD7XqlTfcAbxw&export=download"

clinical_patient <- "./data_clinical_patient.txt"
clinical_sample <- "./data_clinical_sample.txt"
gbm <- "./GBM.uncv2.mRNAseq_RSEM_normalized_log2.txt"

download.file(clinical_patient_url, clinical_patient)
download.file(clinical_sample_url, clinical_sample)
download.file(gbm_url, gbm)

clin.patient <- read.table("./data_clinical_patient.txt", sep = "\t", header = T)
clin.sample <- read.table("./data_clinical_sample.txt", sep = "\t", header = T)
gbm.data <- fread("./GBM.uncv2.mRNAseq_RSEM_normalized_log2.txt")

In [3]:
exprData <- gbm.data
exprData$gene <- exprData$gene %>% gsub("\\|.*", "", .)
exprData <- exprData[-which(exprData$gene == "?"),]

n_occur <- data.frame(table(exprData$gene))
dup_id <- which(exprData$gene %in% n_occur$Var1[n_occur$Freq > 1])
exprData <- as.data.frame(exprData[-dup_id[2],])

exprData <- exprData[,c(1,which(substr(colnames(exprData), 14, 15) == "01" | substr(colnames(exprData), 14, 15) == "11"))]
rownames(exprData) <- exprData$gene
exprData <- exprData[,-1] %>% na.omit()

traitData <- data.frame(sample_id=colnames(exprData), disease_status=substr(colnames(exprData), 14, 15))
traitData$Tumor[which(traitData$disease_status == "01")] <- as.numeric("1")
traitData$Tumor[which(traitData$disease_status == "11")] <- as.numeric("0")
traitData$Normal[which(traitData$disease_status == "01")] <- as.numeric("0")
traitData$Normal[which(traitData$disease_status == "11")] <- as.numeric("1")
rownames(traitData) <- traitData$sample_id
traitData <- traitData[,-c(1,2)]

In [4]:
head(traitData)

,Tumor,Normal
,<dbl>,<dbl>
TCGA-02-0047-01,1,0
TCGA-02-0055-01,1,0
TCGA-02-2483-01,1,0
TCGA-02-2485-01,1,0
TCGA-02-2486-01,1,0
TCGA-06-0125-01,1,0


In [7]:
# enableWGCNAThreads()
allowWGCNAThreads(2)
options(stringsAsFactors = FALSE)

Allowing multi-threading with up to 2 threads.


In [6]:
exprData <- as.data.frame(t(exprData))

In [8]:
head(exprData)

,A1BG,A2BP1,A2LD1,A2M,A4GALT,AAAS,AACS,AADAT,AAGAB,AAK1,⋯,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TCGA-02-0047-01,6.965864,7.934455,5.783258,15.05377,5.180474,8.865786,8.912540,7.867686,10.006052,10.453281,⋯,8.171274,8.023537,8.659774,6.025751,8.477739,10.114814,10.237409,11.91736,10.448212,9.237409
TCGA-02-0055-01,8.613988,7.101725,6.392558,15.38789,8.929958,9.216296,8.309745,6.640820,10.410639,9.965178,⋯,8.518233,8.869007,7.939874,5.413015,8.130871,9.247236,9.852462,13.48701,9.243237,9.488141
TCGA-02-2483-01,8.086679,6.794793,5.110079,14.36212,6.442375,10.104049,8.945699,8.017969,9.920152,9.440623,⋯,9.106813,9.414139,9.386141,4.279434,8.665767,9.758689,10.239186,12.30686,9.700586,9.462370
TCGA-02-2485-01,6.391336,8.006427,6.980547,12.92899,7.278339,9.985571,8.249283,7.569856,10.358494,9.367363,⋯,8.419459,8.782624,8.787592,5.749789,8.099536,10.394156,10.063395,12.31372,10.154215,9.452192
TCGA-02-2486-01,6.758305,2.093662,7.573878,15.32233,5.456228,9.454976,8.614979,7.762038,10.534825,8.983272,⋯,8.716757,7.381015,6.217041,4.986743,7.630600,9.349699,9.429687,12.92926,9.293331,9.042860
TCGA-06-0125-01,7.655826,5.715797,6.828434,13.44798,6.681851,9.696093,8.815706,8.262992,9.689701,9.671797,⋯,8.550061,8.807426,8.585685,5.854317,8.780050,10.176754,10.271101,13.82542,10.399799,9.511926


In [10]:
gsg <- goodSamplesGenes(exprData)
gsg$allOK

 Flagging genes and samples with too many missing values...
  ..step 1


[1] TRUE

In [9]:
dir.create("./Result", showWarnings = TRUE, recursive = FALSE, mode = "0777")

In [ ]:
# plot the sample tree
pdf("./Result/sampleTree.pdf")
sampleTree <- hclust(dist(exprData), method = "average")
options(repr.plot.width=20, repr.plot.height=10)
par(cex = 0.3)
par(mar = c(0,4,2,0))
plot(sampleTree, main = "Sample clustering to detect outliers", sub="", xlab="", cex.lab = 1.5, cex.axis = 1.5, cex.main = 2)
abline(h=215, col="red")
dev.off()

png 
  2

In [ ]:
# cut tree
clust <- cutreeStatic(sampleTree, cutHeight = 215, minSize = 10)
table(clust)
keepSamples <- clust == 1
exprData <- exprData[keepSamples,]
traitData <- traitData[-which(rownames(traitData) == "TCGA-06-2569-01"),]

clust
  0   1 
  1 157 

In [ ]:
# choose a soft-threshold power
powers = c(seq(1, 10, by = 1), seq(12, 20, by = 2))
softThresh <- pickSoftThreshold(exprData, indent = 0)
pdf("./Result/Beta_MeanConnectivity.pdf")
options(repr.plot.width=12, repr.plot.height=20)
par(mfrow = c(1,2))
cex1 = 0.9
plot(softThresh$fitIndices[,1], -sign(softThresh$fitIndices[,3])*softThresh$fitIndices[,2], xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit, signed R^2", type="n", main = paste("Scale independence"));
text(softThresh$fitIndices[,1], -sign(softThresh$fitIndices[,3])*softThresh$fitIndices[,2],labels=powers,cex=cex1,col="blue")
abline(h=0.9, col="red")
plot(softThresh$fitIndices[,1], softThresh$fitIndices[,5], xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",main = paste("Mean connectivity"))
text(softThresh$fitIndices[,1], softThresh$fitIndices[,5], labels=powers, cex=cex1,col="blue")
dev.off()

   Power SFT.R.sq  slope truncated.R.sq  mean.k. median.k. max.k.
1      1    0.101  1.080          0.969 2600.000  2.54e+03 4480.0
2      2    0.219 -0.871          0.949  750.000  6.83e+02 1970.0
3      3    0.658 -1.640          0.967  276.000  2.26e+02 1060.0
4      4    0.760 -2.040          0.962  119.000  8.55e+01  647.0
5      5    0.789 -2.200          0.967   57.800  3.56e+01  423.0
6      6    0.803 -2.250          0.975   30.700  1.59e+01  291.0
7      7    0.814 -2.260          0.982   17.500  7.49e+00  207.0
8      8    0.807 -2.260          0.984   10.500  3.74e+00  151.0
9      9    0.814 -2.200          0.985    6.660  1.92e+00  113.0
10    10    0.817 -2.130          0.983    4.390  1.03e+00   86.0
11    12    0.864 -1.860          0.982    2.100  3.27e-01   51.9
12    14    0.911 -1.720          0.989    1.120  1.15e-01   36.7
13    16    0.931 -1.660          0.992    0.649  4.35e-02   28.6
14    18    0.949 -1.590          0.991    0.402  1.74e-02   22.7
15    20  

png 
  2

In [ ]:
# construction of weighted gene co-expression network
net <- blockwiseModules(exprData, saveTOMs = TRUE, saveTOMFileBase = "./Result/TOM", power = 14, minModuleSize = 30, mergeCutHeight = 0.25)
mergedColors <- labels2colors(net$colors)

pdf("./Result/clusterDendrogram.pdf")
options(repr.plot.width=14, repr.plot.height=20) 
plotDendroAndColors(net$dendrograms[[1]], mergedColors[net$blockGenes[[1]]], "Module\nColors", dendroLabels = FALSE, hang = 0.03, addGuide = TRUE, guideHang = 0.05)
dev.off()

moduleLabels = net$colors
moduleColors = labels2colors(moduleLabels)

geneTree = net$dendrograms[[1]]
nGenes = ncol(exprData)
nSamples = nrow(exprData)
MEs0 = moduleEigengenes(exprData, moduleColors)$eigengenes
MEs = orderMEs(MEs0)
moduleTraitCor = cor(MEs, traitData, use = "p")
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples)

pdf("./Result/module_trait_relationship.pdf")
options(repr.plot.width=14, repr.plot.height=14)
textMatrix = paste(signif(moduleTraitCor, 2), "\n(",signif(moduleTraitPvalue, 1), ")", sep = "")
dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(6, 8.5, 3, 3))
labeledHeatmap(Matrix = moduleTraitCor, xLabels = names(traitData), yLabels = names(MEs), ySymbols = names(MEs), colorLabels = FALSE,
               colors = blueWhiteRed(50),
               textMatrix = textMatrix, setStdMargins = FALSE, cex.text = 0.5, zlim = c(-1,1), main = paste("Module-trait relationships"))
dev.off()

Tumor <- as.data.frame(traitData$Tumor)
Normal <- as.data.frame(traitData$Normal)
names(Tumor) <- "Tumor"
names(Normal) <- "Normal"

# *** Modules with |ME| > 0.5 were selected ***

png 
  2

png 
  2

In [ ]:
# Brown Module
modNames <- substring(names(MEs), 3)
geneModuleMembership <- as.data.frame(cor(exprData, MEs, use = "p"))
MMPvalue <- as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples))
names(geneModuleMembership) <- paste("MM", modNames, sep="")
names(MMPvalue) <- paste("p.MM", modNames, sep="")
geneTraitSignificance <- as.data.frame(cor(exprData, Normal, use = "p"))
GSPvalue <- as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples))
names(geneTraitSignificance) <- paste("GS.", names(Normal), sep="")
names(GSPvalue) <- paste("p.GS.", names(Normal), sep="")

In [ ]:
# identifying genes with high GS and MM
module <- "brown"
column <- match(module, modNames)
moduleGenes <- moduleColors == module
pdf("./Result/scatterBrown.pdf")
par(mfrow = c(1,1))
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]), abs(geneTraitSignificance[moduleGenes, 1]),
                   xlab = paste("Module Membership in", module, "module"),
                   ylab = "Gene significance for Normal",
                   main = paste("Module membership vs. gene significance\n"),
                   cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

png 
  2

In [ ]:
# Yellow Module
modNames <- substring(names(MEs), 3)
geneModuleMembership <- as.data.frame(cor(exprData, MEs, use = "p"))
MMPvalue <- as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples))
names(geneModuleMembership) <- paste("MM", modNames, sep="")
names(MMPvalue) <- paste("p.MM", modNames, sep="")
geneTraitSignificance <- as.data.frame(cor(exprData, Tumor, use = "p"))
GSPvalue <- as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples))
names(geneTraitSignificance) <- paste("GS.", names(Tumor), sep="")
names(GSPvalue) <- paste("p.GS.", names(Tumor), sep="")

In [ ]:
# identifying genes with high GS and MM
module <- "yellow"
column <- match(module, modNames)
moduleGenes <- moduleColors == module
pdf("./Result/scatterYellow.pdf")
par(mfrow = c(1,1))
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]), abs(geneTraitSignificance[moduleGenes, 1]),
                   xlab = paste("Module Membership in", module, "module"),
                   ylab = "Gene significance for Tumor",
                   main = paste("Module membership vs. gene significance\n"),
                   cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

png 
  2

In [ ]:
# Black Module
modNames <- substring(names(MEs), 3)
geneModuleMembership <- as.data.frame(cor(exprData, MEs, use = "p"))
MMPvalue <- as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples))
names(geneModuleMembership) <- paste("MM", modNames, sep="")
names(MMPvalue) <- paste("p.MM", modNames, sep="")
geneTraitSignificance <- as.data.frame(cor(exprData, Normal, use = "p"))
GSPvalue <- as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples))
names(geneTraitSignificance) <- paste("GS.", names(Normal), sep="")
names(GSPvalue) <- paste("p.GS.", names(Normal), sep="")

In [ ]:
# identifying genes with high GS and MM
module <- "black"
column <- match(module, modNames)
moduleGenes <- moduleColors == module
pdf("./Result/scatterBlack.pdf")
par(mfrow = c(1,1))
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]), abs(geneTraitSignificance[moduleGenes, 1]),
                   xlab = paste("Module Membership in", module, "module"),
                   ylab = "Gene significance for Normal",
                   main = paste("Module membership vs. gene significance\n"),
                   cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

# *** Module with r > 0.5 was selected ***
# yellowModuleGenes <- colnames(exprData)[moduleColors == "yellow"]
# blackModuleGenes <- colnames(exprData)[moduleColors == "black"]
brownModuleGenes <- colnames(exprData)[moduleColors == "brown"]

png 
  2

In [ ]:
gbm.data$gene <- gbm.data$gene %>% gsub("\\|.*", "", .)
gbm.data <- as.data.frame(gbm.data[-which(gbm.data$gene == "?"),])
gbm.data <- gbm.data[,c(1,which(substr(colnames(gbm.data), 14, 15) == "01"))]

n_occur <- data.frame(table(gbm.data$gene))
dup_id <- which(gbm.data$gene %in% n_occur$Var1[n_occur$Freq > 1])
gbm.data <- gbm.data[-dup_id[2],] # removing duplicate symbol

rownames(gbm.data) <- gbm.data$gene
gbm.data <- gbm.data[,-1]

clin.sample <- clin.sample[,c(2,3)] %>% na.omit()

Mesenchymal <- clin.sample$SAMPLE_ID[which(clin.sample$EXPRESSION_SUBTYPE == "Mesenchymal")]
NonMesenchymal <- clin.sample$SAMPLE_ID[which(clin.sample$EXPRESSION_SUBTYPE != "Mesenchymal")]

design <- data.table(subtype=rep("", length(colnames(gbm.data))))
design <- as.data.frame(design)
design[which(colnames(gbm.data) %in% Mesenchymal), "subtype"] <- "Mesenchymal"
design[which(colnames(gbm.data) %in% NonMesenchymal), "subtype"] <- "NonMesenchymal"
subtype <- design$subtype
design_matrix <- model.matrix(~ subtype + 0)
design_matrix <- design_matrix[,-1]
colnames(design_matrix) <- c("Mesenchymal", "NonMesenchymal")

gbm.data <- gbm.data[which(rownames(gbm.data) %in% brownModuleGenes),]

ddcor_res = ddcorAll(inputMat = gbm.data, design = design_matrix, compare = c("Mesenchymal", "NonMesenchymal"), heatmapPlot = F)
View(ddcor_res)

cor_res <- getCors(inputMat = gbm.data, design = design_matrix)
pairDCor_res <- pairwiseDCor(cor_res, compare = c("Mesenchymal", "NonMesenchymal"))
dc_TopPairs = dcTopPairs(pairDCor_res, nPairs = 100128, classify = TRUE, adjust = "fdr", plotFdr = F)
dc_TopPairs <- dc_TopPairs[which(dc_TopPairs$pValDiff_adj < 0.05),]
View(dc_TopPairs)
write.csv(dc_TopPairs, "./Result/differentially_co-expression_gene_network.csv") #drawing using cytoscape

pdf("./Result/heatmapPlot.pdf")
options(repr.plot.width = 18, repr.plot.height =20, repr.plot.res = 150)
gbm.data_top = filterGenes(gbm.data, filterTypes = c("central", "dispersion"),
                filterCentralPercentile = 0.65, filterDispersionPercentile = 0.75)

ddcor_res = ddcorAll(inputMat = gbm.data_top, design = design_matrix, compare = c("Mesenchymal", "NonMesenchymal"), heatmapPlot = TRUE, nPairs = "all")
dev.off()

pdf("/content/Result/plotCors.pdf")
options(repr.plot.width = 18, repr.plot.height = 12, repr.plot.res = 150)
plotCors(inputMat = gbm.data, design = design_matrix, compare = c("Mesenchymal", "NonMesenchymal"), geneA = "KIF20A", geneB = "TACC3")
dev.off()

Calculating permutation number 1.

Calculating permutation number 2.

Calculating permutation number 3.

Calculating permutation number 4.

Calculating permutation number 5.

Calculating permutation number 6.

Calculating permutation number 7.

Calculating permutation number 8.

Calculating permutation number 9.

Calculating permutation number 10.

Calculating empirical p-values using the permutation sample statistics.

Sorting the combination of the actual and permuted test statistics.

Finding the proportion of each actual test statistic greater than the permuted test statistics.

Calculating qvalues from the empirical p-values.

Classifying the differential correlation calls.



,Gene1,Gene2,Mesenchymal_cor,Mesenchymal_pVal,NonMesenchymal_cor,NonMesenchymal_pVal,zScoreDiff,pValDiff,empPVals,pValDiff_adj,Classes
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,KIF20A,TACC3,0.92555679,0.000000e+00,0.5951233,4.213563e-11,-5.233454,1.663710e-07,2.654280e-06,0.04470458,+/+
2,CCNA2,TACC3,0.89753257,0.000000e+00,0.4908162,1.630576e-07,-5.129677,2.902401e-07,2.654280e-06,0.04470458,+/+
3,FAM83D,TACC3,0.88072067,0.000000e+00,0.4519165,1.861864e-06,-4.960687,7.024443e-07,5.308560e-06,0.05960611,+/+
4,PLK1,TACC3,0.87915142,2.220446e-16,0.4609545,1.085745e-06,-4.858515,1.182695e-06,2.123424e-05,0.10218190,+/+
5,SPC25,TACC3,0.85477035,1.065814e-14,0.3811259,7.750565e-05,-4.851438,1.225694e-06,2.123424e-05,0.10218190,+/+
6,NCAPG,TACC3,0.90139514,0.000000e+00,0.5490051,2.307432e-09,-4.798217,1.600845e-06,2.123424e-05,0.10218190,+/+
7,TACC3,TPX2,0.92138146,0.000000e+00,0.6264923,1.884271e-12,-4.797214,1.608880e-06,2.123424e-05,0.10218190,+/+
8,CENPA,TACC3,0.88531942,0.000000e+00,0.5032419,7.014512e-08,-4.706761,2.516831e-06,3.715992e-05,0.14443018,+/+
9,GTSE1,TACC3,0.88364752,0.000000e+00,0.5053586,6.055230e-08,-4.648254,3.347564e-06,5.043132e-05,0.14443018,+/+


Classifying the differential correlation calls.



,Gene1,Gene2,corA,corA_pVal,corB,corB_pVal,zScoreDiff,pValDiff,pValDiff_adj,Classes
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,KIF20A,TACC3,0.92555679,0.000000e+00,0.5951233,4.213563e-11,-5.233454,1.663710e-07,0.005467398,+/+
2,CCNA2,TACC3,0.89753257,0.000000e+00,0.4908162,1.630576e-07,-5.129677,2.902401e-07,0.005467398,+/+
3,FAM83D,TACC3,0.88072067,0.000000e+00,0.4519165,1.861864e-06,-4.960687,7.024443e-07,0.008659221,+/+
4,PLK1,TACC3,0.87915142,2.220446e-16,0.4609545,1.085745e-06,-4.858515,1.182695e-06,0.008659221,+/+
5,SPC25,TACC3,0.85477035,1.065814e-14,0.3811259,7.750565e-05,-4.851438,1.225694e-06,0.008659221,+/+
6,NCAPG,TACC3,0.90139514,0.000000e+00,0.5490051,2.307432e-09,-4.798217,1.600845e-06,0.008659221,+/+
7,TACC3,TPX2,0.92138146,0.000000e+00,0.6264923,1.884271e-12,-4.797214,1.608880e-06,0.008659221,+/+
8,CENPA,TACC3,0.88531942,0.000000e+00,0.5032419,7.014512e-08,-4.706761,2.516831e-06,0.011852703,+/+
9,GTSE1,TACC3,0.88364752,0.000000e+00,0.5053586,6.055230e-08,-4.648254,3.347564e-06,0.014013277,+/+


Calculating permutation number 1.

Calculating permutation number 2.

Calculating permutation number 3.

Calculating permutation number 4.

Calculating permutation number 5.

Calculating permutation number 6.

Calculating permutation number 7.

Calculating permutation number 8.

Calculating permutation number 9.

Calculating permutation number 10.

Calculating empirical p-values using the permutation sample statistics.

Sorting the combination of the actual and permuted test statistics.

Finding the proportion of each actual test statistic greater than the permuted test statistics.

Calculating qvalues from the empirical p-values.

Classifying the differential correlation calls.

Plotting heatmap using heatmap.2...



png 
  2

png 
  2

In [ ]:
clin.patient <- clin.patient[,c(1,2)] %>% na.omit()
clin.sample["SAMPLE_ID"] <- lapply(clin.sample["SAMPLE_ID"], function(x) substr(x, 1, 12))
sample.age <- data.table(SAMPLE_ID=clin.sample[which(clin.sample$SAMPLE_ID %in% clin.patient$PATIENT_ID), 1], AGE=clin.patient$AGE)

young <- sample.age$SAMPLE_ID[which(sample.age$AGE <= median(sample.age$AGE))]
old <- sample.age$SAMPLE_ID[which(sample.age$AGE > median(sample.age$AGE))]

Warning message in as.data.table.list(x, keep.rownames = keep.rownames, check.names = check.names, :
“Item 1 has 523 rows but longest item has 542; recycled with remainder.”


In [ ]:
Temp = colnames(gbm.data)
Temp = lapply(Temp, FUN=function(x) substr(x, 1, 12))

In [ ]:
design <- data.table(age=rep("", length(colnames(gbm.data))))
design <- as.data.frame(design)
design[which(Temp %in% young), "age"] <- "young"
design[which(Temp %in% old), "age"] <- "old"

In [ ]:
age <- design$age

In [ ]:
age

[1] "old"   "old"   "young" "young" "old"   "old"   "young" "young" "young"
 [10] "young" "old"   "young" "old"   "old"   "young" "young" "young" "old"  
 [19] "old"   "old"   "old"   "old"   "old"   "old"   "old"   "old"   "young"
 [28] "old"   "young" "young" "old"   "old"   "old"   "young" "young" "old"  
 [37] "young" "old"   "young" "old"   "old"   "old"   "young" "old"   "old"  
 [46] "young" "young" "old"   "young" "young" "young" "old"   "young" "old"  
 [55] "old"   "old"   "old"   "young" "young" "old"   "young" "young" "young"
 [64] "old"   "young" "young" "young" "young" "old"   "young" "old"   "young"
 [73] "young" "old"   "old"   "young" "old"   "young" "old"   "young" "young"
 [82] "old"   "old"   "old"   ""      "young" "young" "young" "old"   "old"  
 [91] "old"   "young" "young" "old"   ""      "young" "old"   "old"   "old"  
[100] "old"   "old"   "young" "young" "old"   "young" "young" "young" "young"
[109] "old"   "old"   "old"   "old"   "old"   "young" "young" "old"   "young"
[118] ""      "old"   ""      "old"   "old"   "old"   "young" "old"   "young"
[127] "old"   "young" "young" "old"   "young" "old"   "old"   "young" "old"  
[136] "young" "old"   "old"   "young" "old"   "old"   "old"   "young" "old"  
[145] "young" "old"   "old"   "old"   "young" "young" "old"   "young" "young"

In [ ]:
design_matrix <- model.matrix(~ age + 0)

In [ ]:
design_matrix

,age,ageold,ageyoung
1,0,1,0
2,0,1,0
3,0,0,1
4,0,0,1
5,0,1,0
6,0,1,0
7,0,0,1
8,0,0,1
9,0,0,1
10,0,0,1


In [ ]:
design_matrix <- design_matrix[,-1]
colnames(design_matrix) <- c("old", "young")

In [ ]:
ddcor_res = ddcorAll(inputMat = gbm.data, design = design_matrix, compare = c("old", "young"), heatmapPlot = F) 
View(ddcor_res)

Calculating permutation number 1.

Calculating permutation number 2.

Calculating permutation number 3.

Calculating permutation number 4.

Calculating permutation number 5.

Calculating permutation number 6.

Calculating permutation number 7.

Calculating permutation number 8.

Calculating permutation number 9.

Calculating permutation number 10.

Calculating empirical p-values using the permutation sample statistics.

Sorting the combination of the actual and permuted test statistics.

Finding the proportion of each actual test statistic greater than the permuted test statistics.

Calculating qvalues from the empirical p-values.

Classifying the differential correlation calls.



,Gene1,Gene2,old_cor,old_pVal,young_cor,young_pVal,zScoreDiff,pValDiff,empPVals,pValDiff_adj,Classes
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,KIF2C,TACC3,0.9038750,0.000000e+00,0.5204570,4.562227e-06,-5.460848,4.738651e-08,0.0000663570,0.9999655,+/+
2,BUB1,TACC3,0.8517111,0.000000e+00,0.4430742,1.375201e-04,-4.687473,2.765997e-06,0.0003636364,0.9999655,+/+
3,TACC3,TPX2,0.8769535,0.000000e+00,0.5469085,1.161354e-06,-4.461891,8.123962e-06,0.0005919044,0.9999655,+/+
4,CDCA8,TACC3,0.8527112,0.000000e+00,0.4771369,3.396034e-05,-4.451593,8.523548e-06,0.0005919044,0.9999655,+/+
5,HJURP,TACC3,0.8536415,0.000000e+00,0.4983408,1.315380e-05,-4.306096,1.661607e-05,0.0008228268,0.9999655,+/+
6,CENPE,TACC3,0.8397971,0.000000e+00,0.4801965,2.973055e-05,-4.156568,3.230644e-05,0.0010723291,0.9999655,+/+
7,AURKB,TACC3,0.7846381,0.000000e+00,0.3478426,3.404268e-03,-4.139238,3.484615e-05,0.0011068348,0.9999655,+/+
8,CKAP2L,TACC3,0.8262447,0.000000e+00,0.4480674,1.130553e-04,-4.136655,3.524048e-05,0.0011068348,0.9999655,+/+
9,NCAPH,TACC3,0.8527448,0.000000e+00,0.5321912,2.522189e-06,-4.011620,6.030340e-05,0.0014226941,0.9999655,+/+


In [ ]:
cor_res <- getCors(inputMat = gbm.data, design = design_matrix)
pairDCor_res <- pairwiseDCor(cor_res, compare = c("old", "young"))
dc_TopPairs = dcTopPairs(pairDCor_res, nPairs = 100128, classify = TRUE, adjust = "fdr", plotFdr = F)
dc_TopPairs <- dc_TopPairs[which(dc_TopPairs$pValDiff_adj < 0.05),]
View(dc_TopPairs)
write.csv(dc_TopPairs, "/content/Result/AGE_differentially_co-expression_gene_network.csv") #drawing using cytoscape

pdf("/content/Result/AGE_heatmapPlot.pdf")
options(repr.plot.width = 18, repr.plot.height =20, repr.plot.res = 150)
gbm.data_top = filterGenes(gbm.data, filterTypes = c("central", "dispersion"),
                filterCentralPercentile = 0.65, filterDispersionPercentile = 0.75)

ddcor_res = ddcorAll(inputMat = gbm.data_top, design = design_matrix, compare = c("old", "young"), heatmapPlot = TRUE, nPairs = "all")
dev.off()

pdf("/content/Result/AGE_plotCors.pdf")
options(repr.plot.width = 18, repr.plot.height = 12, repr.plot.res = 150)
plotCors(inputMat = gbm.data, design = design_matrix, compare = c("old", "young"), geneA = "CDCA8", geneB = "FAM83D")
dev.off()

Classifying the differential correlation calls.



,Gene1,Gene2,corA,corA_pVal,corB,corB_pVal,zScoreDiff,pValDiff,pValDiff_adj,Classes
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,KIF2C,TACC3,0.903875,0,0.520457,4.562227e-06,-5.460848,4.738651e-08,0.001785287,+/+


Calculating permutation number 1.

Calculating permutation number 2.

Calculating permutation number 3.

Calculating permutation number 4.

Calculating permutation number 5.

Calculating permutation number 6.

Calculating permutation number 7.

Calculating permutation number 8.

Calculating permutation number 9.

Calculating permutation number 10.

Calculating empirical p-values using the permutation sample statistics.

Sorting the combination of the actual and permuted test statistics.

Finding the proportion of each actual test statistic greater than the permuted test statistics.

Calculating qvalues from the empirical p-values.

Classifying the differential correlation calls.

Plotting heatmap using heatmap.2...



png 
  2

png 
  2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')